In [1]:
#Import libraries
from pyhive import presto
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import h3
from datetime import datetime
from functools import reduce
import json
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [2]:
#Define presto credentials
presto_host = 'presto-gateway.serving.data.plectrum.dev'
presto_port = '80'
username = 'aditya.bhattar@rapido.bike'

#Create connection to presto host
connection = presto.connect(presto_host,presto_port,username = username)

In [15]:
#Load the hexes in zone and holdout
hexes = pd.read_excel('auto_segments_bangalore.xlsx', sheet_name = 'hsr_lower')
hexes = hexes.rename(columns = {'uid':'pickup_location_hex_8', 'segment':'Segment'})
hyderabad_new_zone = hexes['pickup_location_hex_8'].unique().tolist()
hexes.head()

,pickup_location_hex_8,Segment,Label
0,88618924e7fffff,S1,Zone
1,88618925cbfffff,S1,Zone
2,88618924bdfffff,S1,Holdout
3,8861892405fffff,S1,Zone
4,8860145a0dfffff,S1,Holdout


In [16]:
print(hyderabad_new_zone)

['88618924e7fffff', '88618925cbfffff', '88618924bdfffff', '8861892405fffff', '8860145a0dfffff', '886189243bfffff', '886014586dfffff', '8861892cc9fffff', '8861892ec9fffff', '8861892415fffff', '8861892c19fffff', '8861892439fffff', '886189263dfffff', '8861892437fffff', '8861892433fffff', '8861892cddfffff', '8860145ac9fffff', '886189246dfffff', '88618920adfffff', '8860145841fffff', '886189246bfffff', '8861892c1dfffff', '8860145a25fffff', '886189243dfffff', '88618925ddfffff', '8860145b15fffff', '8861892407fffff', '8861892631fffff', '8861892c55fffff', '8861892607fffff', '88601451e5fffff', '88601459a9fffff', '8860145933fffff', '8861892623fffff', '886014586bfffff', '886014c939fffff', '8861892615fffff', '8861892cd9fffff', '886189268dfffff', '8860145137fffff', '8860145149fffff', '8860145833fffff', '8860145ad3fffff', '8861892693fffff', '8860145927fffff', '8861893483fffff', '88601451adfffff', '88601696e3fffff', '8861892f05fffff', '8861892561fffff', '8861892461fffff', '8860145323fffff', '8861892403

In [17]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, city_name, hhmmss, hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason, eta, map_riders, pickup_location_hex_8
from orders.order_logs_snapshot
where yyyymmdd >= '20230420'
and yyyymmdd <= '20230501'
and city_name in ('Bangalore')
and service_obj_service_name = 'Auto'
and pickup_location_hex_8 in ('88618924e7fffff', '88618925cbfffff', '88618924bdfffff', '8861892405fffff', '8860145a0dfffff', '886189243bfffff', '886014586dfffff', '8861892cc9fffff', '8861892ec9fffff', '8861892415fffff', '8861892c19fffff', '8861892439fffff', '886189263dfffff', '8861892437fffff', '8861892433fffff', '8861892cddfffff', '8860145ac9fffff', '886189246dfffff', '88618920adfffff', '8860145841fffff', '886189246bfffff', '8861892c1dfffff', '8860145a25fffff', '886189243dfffff', '88618925ddfffff', '8860145b15fffff', '8861892407fffff', '8861892631fffff', '8861892c55fffff', '8861892607fffff', '88601451e5fffff', '88601459a9fffff', '8860145933fffff', '8861892623fffff', '886014586bfffff', '886014c939fffff', '8861892615fffff', '8861892cd9fffff', '886189268dfffff', '8860145137fffff', '8860145149fffff', '8860145833fffff', '8860145ad3fffff', '8861892693fffff', '8860145927fffff', '8861893483fffff', '88601451adfffff', '88601696e3fffff', '8861892f05fffff', '8861892561fffff', '8861892461fffff', '8860145323fffff', '8861892403fffff', '8861892429fffff', '8861892cd3fffff', '8861892445fffff', '8861892447fffff', '88618921d7fffff', '88601458c9fffff', '8861892469fffff', '8861892519fffff', '8861892e15fffff', '8861892c81fffff', '88618924d3fffff', '8861892465fffff', '8860145911fffff', '8861892e3bfffff', '886189241dfffff', '8861892687fffff', '8861892401fffff', '88618920d1fffff', '8861892511fffff', '8861892f51fffff', '88618924d1fffff', '8861892017fffff', '886189275dfffff', '8861892cdbfffff', '8861892f55fffff', '8861892711fffff', '8861892c23fffff', '8861892703fffff', '88618920c7fffff', '8860169651fffff', '8860144b69fffff', '8861892abbfffff', '8861892f5bfffff', '886014583bfffff', '8860169741fffff', '88618926e3fffff', '8861892011fffff', '886189359dfffff', '8861892035fffff', '8861892d85fffff', '8861892515fffff', '8861892507fffff', '886189250bfffff', '8861892c27fffff', '8860145ab1fffff', '88618926d5fffff', '8861892113fffff', '886016975bfffff', '8861892695fffff', '8861892ab3fffff', '886016906bfffff', '8860145ab7fffff', '88618925cdfffff', '8861892599fffff', '88618924e5fffff', '88618925d9fffff', '8861892431fffff', '88618924c5fffff', '886189242bfffff', '8861892557fffff', '8861892423fffff', '88618925dbfffff', '88618925d1fffff', '8861892421fffff', '8860145a45fffff', '8861892463fffff', '8861892427fffff', '88618924adfffff', '88618925d3fffff', '8861892663fffff', '8861892487fffff', '88618924edfffff', '88618925c9fffff', '886189240dfffff', '8861892425fffff', '8861892493fffff', '8861892435fffff', '886016966bfffff', '8860145ac5fffff', '8860145901fffff', '88618935bbfffff', '8860145ad1fffff', '8860145b25fffff', '8861892ddbfffff', '886189266bfffff', '8861892db9fffff', '8861892a9bfffff', '8860145825fffff', '8860145a03fffff', '8860145ae5fffff', '8860145325fffff', '88601696d5fffff', '886014c935fffff', '8860145349fffff', '886016968dfffff', '8860145331fffff', '8860169685fffff', '8861892ecbfffff', '8861892e9bfffff', '886189242dfffff', '886189256dfffff', '8861892553fffff', '8861892467fffff', '88618920a5fffff', '886189255bfffff', '8861892ed7fffff', '88618925b3fffff', '8861892c9dfffff', '8861892e55fffff', '8861892533fffff', '8861892c95fffff', '8861892095fffff', '886014594dfffff', '88618926c5fffff', '886014c903fffff', '8861892085fffff', '8860169649fffff', '886016965dfffff', '88618921c9fffff', '88618920c1fffff', '8861892709fffff', '8861892187fffff', '88618921a1fffff', '88618922a7fffff', '886189251dfffff', '8861892c21fffff', '886189229dfffff', '8860144b51fffff', '886014c9adfffff', '8860144a65fffff', '8861892287fffff')
and hour in ('17', '18', '19', '20', '21')
"""

#Load data into pandas table
df_copy = pd.read_sql(q, connection)

In [18]:
#Create a copy of the df retrived from presto
df = df_copy.copy()
df = df[df['pickup_location_hex_8'].isin(hyderabad_new_zone)]
df.head()

,order_id,accept_to_pickup_distance,city_name,hhmmss,hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,map_riders,pickup_location_hex_8
0,6443ef5c30bcb25863b2d3a3,NaN,Bangalore,195948,19,Auto,19:59,6,20230422,expired,None,4.0,"[""635f8653d58af72490de3f79"",""6198e7be3f25c3232...",8861892429fffff
1,644fb76d6aa7f322ac375e47,0.561,Bangalore,182821,18,Auto,18:00,1,20230501,dropped,None,3.0,"[""636a0b88cf056dad8c22bdf4"",""5bb865b27e7bc30a3...",8861892553fffff
2,644fb76e6aa7f322ac375e48,NaN,Bangalore,182822,18,Auto,18:00,1,20230501,expired,None,0.0,"[""5d96150ec2153814dfc01662"",""62e1ff185b3d20890...",8861892423fffff
3,644fb7912a803956b6a0a93c,NaN,Bangalore,182857,18,Auto,18:00,1,20230501,expired,None,2.0,"[""5e5daef25876db43584520eb"",""62cececfe9e2514d2...",886189255bfffff
4,644fb7e9b3a25d325488489c,NaN,Bangalore,183025,18,Auto,18:00,1,20230501,expired,None,2.0,"[""611602368dddcb97456d956b"",""613f122b7f5ff7ba3...",88618925cbfffff


In [19]:
#View the dataset
df_captain = df.copy()
df_captain = df_captain[df_captain['pickup_location_hex_8'].isin(hyderabad_new_zone)]
df_captain.head()

,order_id,accept_to_pickup_distance,city_name,hhmmss,hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,map_riders,pickup_location_hex_8
0,6443ef5c30bcb25863b2d3a3,NaN,Bangalore,195948,19,Auto,19:59,6,20230422,expired,None,4.0,"[""635f8653d58af72490de3f79"",""6198e7be3f25c3232...",8861892429fffff
1,644fb76d6aa7f322ac375e47,0.561,Bangalore,182821,18,Auto,18:00,1,20230501,dropped,None,3.0,"[""636a0b88cf056dad8c22bdf4"",""5bb865b27e7bc30a3...",8861892553fffff
2,644fb76e6aa7f322ac375e48,NaN,Bangalore,182822,18,Auto,18:00,1,20230501,expired,None,0.0,"[""5d96150ec2153814dfc01662"",""62e1ff185b3d20890...",8861892423fffff
3,644fb7912a803956b6a0a93c,NaN,Bangalore,182857,18,Auto,18:00,1,20230501,expired,None,2.0,"[""5e5daef25876db43584520eb"",""62cececfe9e2514d2...",886189255bfffff
4,644fb7e9b3a25d325488489c,NaN,Bangalore,183025,18,Auto,18:00,1,20230501,expired,None,2.0,"[""611602368dddcb97456d956b"",""613f122b7f5ff7ba3...",88618925cbfffff


In [20]:
#Filter only on the hexes needed
df_filter_level1 = df
df_filter_level1['count'] = 1
df_filter_level1['date'] = df_filter_level1['yyyymmdd'].astype(str).apply(lambda x: x[6:])

In [21]:
#Create a new column for cancel_reason to identify OCARA
df_filter_level1['new_cancel_reason'] = np.where(df_filter_level1['order_status'] == 'customerCancelled',
 np.where(df_filter_level1['cancel_reason'] == 'order cancelled before rider accepted', 'COBRA',
 np.where(df_filter_level1['cancel_reason'] == 'Order cancelled before rider was mapped', 'COBRM', 'OCARA')), 'no_cancellation')

In [22]:
#Check if the order is expiry_mapped or expiry_unmapped
df_filter_level1['expiry_type'] = np.where(df_filter_level1['order_status'] == 'expired',
 np.where(df_filter_level1['map_riders'] == '[]', 'expiry_unmapped', 'expiry_mapped'), 'no_expiry')
df_filter_level1['final_order_status'] = np.where(df_filter_level1['order_status'] == 'customerCancelled', df_filter_level1['new_cancel_reason'],
np.where(df_filter_level1['order_status'] == 'expired', df_filter_level1['expiry_type'], df_filter_level1['order_status']))

In [23]:
#Add segment and label to the datasets
df_filter_level1 = df_filter_level1.merge(hexes, on = 'pickup_location_hex_8', how = 'left')
df_captain = df_captain.merge(hexes, on = 'pickup_location_hex_8', how = 'left')
df_filter_level1.head()

,order_id,accept_to_pickup_distance,city_name,hhmmss,hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,map_riders,pickup_location_hex_8,count,date,new_cancel_reason,expiry_type,final_order_status,Segment,Label
0,6443ef5c30bcb25863b2d3a3,NaN,Bangalore,195948,19,Auto,19:59,6,20230422,expired,None,4.0,"[""635f8653d58af72490de3f79"",""6198e7be3f25c3232...",8861892429fffff,1,22,no_cancellation,expiry_mapped,expiry_mapped,S2,Zone
1,644fb76d6aa7f322ac375e47,0.561,Bangalore,182821,18,Auto,18:00,1,20230501,dropped,None,3.0,"[""636a0b88cf056dad8c22bdf4"",""5bb865b27e7bc30a3...",8861892553fffff,1,01,no_cancellation,no_expiry,dropped,S4,Zone
2,644fb76e6aa7f322ac375e48,NaN,Bangalore,182822,18,Auto,18:00,1,20230501,expired,None,0.0,"[""5d96150ec2153814dfc01662"",""62e1ff185b3d20890...",8861892423fffff,1,01,no_cancellation,expiry_mapped,expiry_mapped,S3,Zone
3,644fb7912a803956b6a0a93c,NaN,Bangalore,182857,18,Auto,18:00,1,20230501,expired,None,2.0,"[""5e5daef25876db43584520eb"",""62cececfe9e2514d2...",886189255bfffff,1,01,no_cancellation,expiry_mapped,expiry_mapped,S4,Zone
4,644fb7e9b3a25d325488489c,NaN,Bangalore,183025,18,Auto,18:00,1,20230501,expired,None,2.0,"[""611602368dddcb97456d956b"",""613f122b7f5ff7ba3...",88618925cbfffff,1,01,no_cancellation,expiry_mapped,expiry_mapped,S1,Zone


In [24]:
#Metrics_pivot
metrics_pivot = pd.pivot_table(df_filter_level1, index = ['yyyymmdd', 'hour', 'Segment', 'Label'], columns = 'final_order_status',
 values = 'count', aggfunc = 'sum', fill_value=0).reset_index().rename_axis(None, axis=1)
metrics_pivot['total'] = metrics_pivot[['COBRA', 'COBRM', 'OCARA', 'aborted', 'dropped', 'expiry_mapped', 'expiry_unmapped']].sum(axis = 1)
metrics_pivot['OCARA%'] = round(metrics_pivot['OCARA']/metrics_pivot['total'], 4)
metrics_pivot['COBRA%'] = round(metrics_pivot['COBRA']/metrics_pivot['total'], 4)
metrics_pivot['COBRM%'] = round(metrics_pivot['COBRM']/metrics_pivot['total'], 4)
metrics_pivot['G2N'] = round(metrics_pivot['dropped']/metrics_pivot['total'], 4)
metrics_pivot['expiry_mapped%'] = round(metrics_pivot['expiry_mapped']/metrics_pivot['total'], 4)
if 'expiry_unmapped' in metrics_pivot.columns:
    metrics_pivot['Stockout%'] = round(metrics_pivot['expiry_unmapped']/metrics_pivot['total'], 4)
else:
    metrics_pivot['Stockout%'] = 0
metrics_df = metrics_pivot[['yyyymmdd', 'hour', 'Segment', 'Label', 'OCARA%', 'COBRA%', 'COBRM%', 'G2N', 'expiry_mapped%', 'Stockout%', 'total']]
metrics_df.head()

,yyyymmdd,hour,Segment,Label,OCARA%,COBRA%,COBRM%,G2N,expiry_mapped%,Stockout%,total
0,20230420,17,S1,Holdout,0.1141,0.2886,0.0034,0.4262,0.1510,0.0134,298
1,20230420,17,S1,Zone,0.1198,0.2262,0.0000,0.5513,0.1027,0.0000,526
2,20230420,17,S2,Holdout,0.1050,0.3094,0.0000,0.3149,0.2597,0.0110,181
3,20230420,17,S2,Zone,0.1395,0.1395,0.0000,0.6357,0.0698,0.0000,129
4,20230420,17,S3,Holdout,0.1258,0.2742,0.0000,0.3888,0.2045,0.0045,445


In [25]:
#Get the average ETA and FM
fm_eta = df_filter_level1.groupby(['yyyymmdd', 'hour', 'Segment', 'Label'])[['accept_to_pickup_distance', 'eta']].median().reset_index()
fm_eta = fm_eta.rename(columns = {'accept_to_pickup_distance':'FM'})
metrics_df = metrics_df.merge(fm_eta, on = ['yyyymmdd', 'hour', 'Segment', 'Label'], how = 'left')
metrics_df.head()

,yyyymmdd,hour,Segment,Label,OCARA%,COBRA%,COBRM%,G2N,expiry_mapped%,Stockout%,total,FM,eta
0,20230420,17,S1,Holdout,0.1141,0.2886,0.0034,0.4262,0.1510,0.0134,298,0.6955,2.0
1,20230420,17,S1,Zone,0.1198,0.2262,0.0000,0.5513,0.1027,0.0000,526,0.6450,2.0
2,20230420,17,S2,Holdout,0.1050,0.3094,0.0000,0.3149,0.2597,0.0110,181,0.8560,2.0
3,20230420,17,S2,Zone,0.1395,0.1395,0.0000,0.6357,0.0698,0.0000,129,0.6800,3.0
4,20230420,17,S3,Holdout,0.1258,0.2742,0.0000,0.3888,0.2045,0.0045,445,0.8720,3.0


In [26]:
metrics_df.to_csv('fm_lower.csv', index = False)


In [14]:
#Get the number of captains by cluster
#Make columns for second and minute and date
df_captain['second'] = df_captain['hhmmss'].astype(str).apply(lambda x: x[4:])
df_captain['minute'] = df_captain['hhmmss'].astype(str).apply(lambda x: x[2:4])
df_captain['date'] = df_captain['yyyymmdd'].astype(str).apply(lambda x: x[6:])
df_captain['map_riders'] = df_captain['map_riders'].apply(lambda x: json.loads(x))
df_captain = df_captain[df_captain['map_riders'].apply(lambda x: len(x)) > 0]
df_captain['15_sec_batch'] = pd.cut(df_captain['second'].astype(int), bins = [-1, 15, 30, 45, 61], labels = ['0-15', '15-30', '30-45', '45-60'])

#Get the riders mapped in 20_sec_windows
df_captains_20_sec = df_captain.groupby(['yyyymmdd', 'hour', 'Segment', 'Label', 'minute', '15_sec_batch'])['map_riders'].apply(list).reset_index()

#Get the number of captains in a single list
map_riders = []
df_captains_20_sec = df_captains_20_sec.dropna()
for x in df_captains_20_sec['map_riders'].values:
    riders = [item for sublist in x for item in sublist]
    map_riders.append(riders)

unique_captains = []
for x in map_riders:
    unique_captains_list = []
    for y in x:
        if y in unique_captains_list:
            continue
        else:
            unique_captains_list.append(y)
    unique_captains.append(unique_captains_list)

df_captains_20_sec['total_captains'] = unique_captains

#Get the number of captains in each batch
df_captains_20_sec['number_of_captains'] = df_captains_20_sec['total_captains'].apply(lambda x: len(x))

#Get the average number of captains coming in each 20_second_window
df_20_sec_average = df_captains_20_sec.groupby(['yyyymmdd', 'hour', 'Segment', 'Label'])['number_of_captains'].median().reset_index()

#Merge the hourly captains and 20_sec_captains with the metrics table
metrics_df = metrics_df.merge(df_20_sec_average, on = ['yyyymmdd', 'hour', 'Segment', 'Label'], how = 'left')
metrics_df.head()

,yyyymmdd,hour,Segment,Label,OCARA%,COBRA%,COBRM%,G2N,expiry_mapped%,Stockout%,total,FM,eta,number_of_captains
0,20230420,17,balanced-high,Holdout,0.0778,0.3346,0.0000,0.4747,0.1128,0.0000,257,24.310947,3.070039,46.5
1,20230420,17,balanced-high,Zone,0.1168,0.1825,0.0000,0.6095,0.0876,0.0000,274,27.224927,2.722628,72.0
2,20230420,17,balanced-low,Holdout,0.1373,0.2745,0.0000,0.2549,0.3137,0.0000,51,1.125160,2.823529,13.0
3,20230420,17,in-high,Holdout,0.0954,0.2977,0.0016,0.3865,0.2138,0.0016,608,0.856027,2.861842,72.0
4,20230420,17,in-high,Zone,0.1222,0.2379,0.0011,0.5080,0.1275,0.0021,933,9.423077,2.780279,139.0


In [15]:
#Query data from order_logs_immutable for calculating TTC

q = """
select order_id, accept_to_pickup_distance, captain_location_hex_8, captain_location_latitude,
captain_location_longitude, city_name, customer_location_hex_8, drop_location_hex_8, drop_location_latitude, drop_location_longitude, customer_location_latitude, 
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason, updated_epoch, event_type, pickup_location_hex_8
from orders.order_logs_immutable
where yyyymmdd >= '20230420'
and yyyymmdd <= '20230501'
and city_name in ('Bangalore')
and service_obj_service_name = 'Auto'
and event_type in ('customer_cancelled', 'order_requested', 'accepted')
and pickup_location_hex_8 in ('88618924bbfffff', '886189260bfffff', '8860145913fffff', '88618924b3fffff', '8860145823fffff', '8860169659fffff', '886014595dfffff',
 '8860145a07fffff', '8860145847fffff', '8860169607fffff', '8861892c81fffff', '88601451e1fffff', '8861892687fffff', '88618921e3fffff', '8861892187fffff',
   '88601451e5fffff', '8860145acdfffff', '88618920ebfffff', '886189203dfffff', '8860145a83fffff', '8860145a87fffff', '886189208dfffff', '8860169645fffff',
     '88601696dbfffff', '8861892debfffff', '8861892495fffff', '8860145125fffff', '8861892c3bfffff', '88618921c5fffff', '8860169621fffff', '8860169619fffff',
       '8860145aadfffff', '8860145921fffff', '88618934e3fffff', '886014c905fffff', '8860145907fffff', '88618921cdfffff', '8861892281fffff', '8860145101fffff',
         '8860145ab5fffff', '8861892117fffff', '8861893495fffff', '88601690c7fffff', '886016968dfffff', '8860145995fffff', '886189228dfffff', '886016944bfffff',
           '88618934b1fffff', '8861892f65fffff', '8860169687fffff', '8860169743fffff', '8860145115fffff', '8861892691fffff', '8860169655fffff', '8860145359fffff',
             '8861892461fffff', '8860145b3bfffff', '88618925ddfffff', '8861892427fffff', '8861892c4bfffff', '88618925e9fffff', '8861892433fffff', '8861892401fffff',
               '886189242bfffff', '8861892423fffff', '88618925c9fffff', '88618924e7fffff', '8861892437fffff', '8861892439fffff', '8861892447fffff',
                 '8861892415fffff', '8861892c53fffff', '8861892469fffff', '886189243bfffff', '886189246bfffff', '886189243dfffff', '8861892e03fffff',
                   '886189246dfffff', '88618925cbfffff', '8861892e33fffff', '886189240bfffff', '8861892403fffff', '8860145b43fffff', '8861892e5dfffff',
                     '8861892463fffff', '88618920adfffff', '88618924adfffff', '8861892563fffff', '8861892cd3fffff', '8861892721fffff', '8860145a59fffff',
                       '8861892429fffff', '8861892eb3fffff', '8861892e17fffff', '8861892e61fffff', '886189252bfffff', '8861892445fffff', '8861892431fffff',
                         '8861892e67fffff', '8860145b19fffff', '8860145a55fffff', '8861892603fffff', '886189240dfffff', '8861892519fffff', '8861892405fffff',
                           '8861892407fffff', '8861892451fffff', '8861892e05fffff', '8861892717fffff', '8860145945fffff', '8861892ee7fffff', '8861892e45fffff',
                             '8861892c09fffff', '8861892465fffff', '88618926c1fffff', '8861892085fffff', '8861892625fffff', '88618920d3fffff', '886016975bfffff',
                               '8861892c69fffff', '886014534dfffff', '8860144b43fffff', '8860145acbfffff', '886014592bfffff', '8861892cb5fffff', '8861892ca9fffff',
                                 '8861892da3fffff', '8860145133fffff', '8861892db9fffff', '88618926e1fffff', '8860145951fffff', '88601458c5fffff', '8860145831fffff',
                                   '886014c925fffff', '8861892dc5fffff', '8861892dc7fffff', '88618926d1fffff', '88618935b7fffff', '8861892467fffff',
                                     '8861892591fffff', '88618925d9fffff', '88618925dbfffff', '8861892425fffff', '88618924e5fffff', '88618924edfffff',
                                       '8861892e1bfffff', '8861892435fffff', '8860145b1dfffff', '88601451adfffff', '8860145365fffff', '886014c9adfffff',
                                         '88618922b3fffff', '8860145ac9fffff', '8861892ca5fffff', '886014536bfffff', '88618935b1fffff', '8860145b37fffff',
                                           '8860145919fffff', '886014c9a9fffff', '886189269dfffff', '8860145a15fffff', '88618934b3fffff', '8860144b4dfffff',
                                             '88601451e7fffff', '8861892697fffff', '886014c923fffff', '8860145aedfffff', '886014c915fffff', '8860145341fffff',
                                               '8860145117fffff', '8861892293fffff', '8860169719fffff', '8860145363fffff', '88601459b5fffff', '8860145ae1fffff',
                                                 '886014c92dfffff', '88601696e3fffff', '8861892c63fffff', '8860145113fffff', '8860144a6dfffff', '8860145315fffff',
                                                   '8860144b55fffff', '886014c937fffff', '8860145abbfffff', '8860145b4bfffff', '8861892e91fffff', '88618925d3fffff',
                                                     '8861892621fffff', '8860145b49fffff', '88618925d1fffff', '886189255bfffff', '8861892545fffff', '88618925d7fffff',
                                                       '8861892e0dfffff', '8861892421fffff', '886189258dfffff', '88618925b5fffff', '8861892553fffff',
                                                         '886189242dfffff', '88618925b7fffff', '8860145949fffff', '8861892e41fffff', '886189219bfffff',
                                                           '886016965dfffff', '8861892539fffff', '886189265bfffff', '8861892505fffff', '88618934a3fffff',
                                                             '8861892501fffff', '8860169681fffff')
and hour in ('17', '18', '19', '20', '21')
"""

#Load data into pandas table
df_ttc_copy = pd.read_sql(q, connection)

In [16]:
#Load and view dataset
df_ttc = df_ttc_copy.copy()
df_ttc = df_ttc[df_ttc['pickup_location_hex_8'].isin(hyderabad_new_zone)]
df_ttc['timestamp'] = df_ttc['updated_epoch'].apply(lambda x: datetime.fromtimestamp(x/1000))
df_ttc = df_ttc.sort_values(by = 'timestamp')
df_ttc.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,updated_epoch,event_type,pickup_location_hex_8,timestamp
149287,644122381c6d14472f498524,NaN,,NaN,NaN,Bangalore,88618925d3fffff,8861892589fffff,12.930696,77.603659,12.918210,77.591995,170000,17,1700,Auto,17:00,4,20230420,requested,None,1681990200725,order_requested,88618925d3fffff,2023-04-20 17:00:00.725
229171,6441223959513b224af3f391,NaN,,NaN,NaN,Bangalore,8860145a4dfffff,886189258dfffff,12.941111,77.608582,12.927545,77.567802,170001,17,1700,Auto,17:00,4,20230420,requested,None,1681990201911,order_requested,8860145b49fffff,2023-04-20 17:00:01.911
53283,6441223a3382915e31cf0edd,NaN,,NaN,NaN,Bangalore,88618925d9fffff,8861892ec3fffff,12.981894,77.641900,12.914279,77.601508,170002,17,1700,Auto,17:00,4,20230420,requested,None,1681990202293,order_requested,88618925d9fffff,2023-04-20 17:00:02.293
57290,6441223cf4689d1c09459f0e,NaN,,NaN,NaN,Bangalore,886189219bfffff,88618921d5fffff,12.976319,77.748917,12.981582,77.727074,170004,17,1700,Auto,17:00,4,20230420,requested,None,1681990204802,order_requested,886189219bfffff,2023-04-20 17:00:04.802
115888,6441223e1c6d14472f49852f,NaN,,NaN,NaN,Bangalore,88618924e5fffff,88618925c5fffff,12.936004,77.621627,12.905972,77.595413,170006,17,1700,Auto,17:00,4,20230420,requested,None,1681990206165,order_requested,88618924e5fffff,2023-04-20 17:00:06.165


In [17]:
#Create a column to identify OCARA, COBRA and COBRM
df_ttc['new_cancel_reason'] = np.where(df_ttc['event_type'] == 'customer_cancelled',
 np.where(df_ttc['cancel_reason'] == 'order cancelled before rider accepted', 'COBRA',
 np.where(df_ttc['cancel_reason'] == 'Order cancelled before rider was mapped', 'COBRM', 'OCARA')), 'no_cancellation')
df_ttc.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,updated_epoch,event_type,pickup_location_hex_8,timestamp,new_cancel_reason
149287,644122381c6d14472f498524,NaN,,NaN,NaN,Bangalore,88618925d3fffff,8861892589fffff,12.930696,77.603659,12.918210,77.591995,170000,17,1700,Auto,17:00,4,20230420,requested,None,1681990200725,order_requested,88618925d3fffff,2023-04-20 17:00:00.725,no_cancellation
229171,6441223959513b224af3f391,NaN,,NaN,NaN,Bangalore,8860145a4dfffff,886189258dfffff,12.941111,77.608582,12.927545,77.567802,170001,17,1700,Auto,17:00,4,20230420,requested,None,1681990201911,order_requested,8860145b49fffff,2023-04-20 17:00:01.911,no_cancellation
53283,6441223a3382915e31cf0edd,NaN,,NaN,NaN,Bangalore,88618925d9fffff,8861892ec3fffff,12.981894,77.641900,12.914279,77.601508,170002,17,1700,Auto,17:00,4,20230420,requested,None,1681990202293,order_requested,88618925d9fffff,2023-04-20 17:00:02.293,no_cancellation
57290,6441223cf4689d1c09459f0e,NaN,,NaN,NaN,Bangalore,886189219bfffff,88618921d5fffff,12.976319,77.748917,12.981582,77.727074,170004,17,1700,Auto,17:00,4,20230420,requested,None,1681990204802,order_requested,886189219bfffff,2023-04-20 17:00:04.802,no_cancellation
115888,6441223e1c6d14472f49852f,NaN,,NaN,NaN,Bangalore,88618924e5fffff,88618925c5fffff,12.936004,77.621627,12.905972,77.595413,170006,17,1700,Auto,17:00,4,20230420,requested,None,1681990206165,order_requested,88618924e5fffff,2023-04-20 17:00:06.165,no_cancellation


In [18]:
#Filter the df only zone1
df_ttc_filter = df_ttc
df_ttc_filter = df_ttc_filter.merge(hexes, on = 'pickup_location_hex_8', how = 'left')
df_ttc_filter.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,updated_epoch,event_type,pickup_location_hex_8,timestamp,new_cancel_reason,Segment,Label
0,644122381c6d14472f498524,NaN,,NaN,NaN,Bangalore,88618925d3fffff,8861892589fffff,12.930696,77.603659,12.918210,77.591995,170000,17,1700,Auto,17:00,4,20230420,requested,None,1681990200725,order_requested,88618925d3fffff,2023-04-20 17:00:00.725,no_cancellation,out-high,Zone
1,6441223959513b224af3f391,NaN,,NaN,NaN,Bangalore,8860145a4dfffff,886189258dfffff,12.941111,77.608582,12.927545,77.567802,170001,17,1700,Auto,17:00,4,20230420,requested,None,1681990201911,order_requested,8860145b49fffff,2023-04-20 17:00:01.911,no_cancellation,out-high,Holdout
2,6441223a3382915e31cf0edd,NaN,,NaN,NaN,Bangalore,88618925d9fffff,8861892ec3fffff,12.981894,77.641900,12.914279,77.601508,170002,17,1700,Auto,17:00,4,20230420,requested,None,1681990202293,order_requested,88618925d9fffff,2023-04-20 17:00:02.293,no_cancellation,balanced-high,Holdout
3,6441223cf4689d1c09459f0e,NaN,,NaN,NaN,Bangalore,886189219bfffff,88618921d5fffff,12.976319,77.748917,12.981582,77.727074,170004,17,1700,Auto,17:00,4,20230420,requested,None,1681990204802,order_requested,886189219bfffff,2023-04-20 17:00:04.802,no_cancellation,out-high,Holdout
4,6441223e1c6d14472f49852f,NaN,,NaN,NaN,Bangalore,88618924e5fffff,88618925c5fffff,12.936004,77.621627,12.905972,77.595413,170006,17,1700,Auto,17:00,4,20230420,requested,None,1681990206165,order_requested,88618924e5fffff,2023-04-20 17:00:06.165,no_cancellation,balanced-high,Zone


In [19]:
#Filter df on the columns needed
df_filter = df_ttc_filter[['order_id', 'pickup_location_hex_8', 'hour', 'yyyymmdd', 'timestamp', 'event_type', 'new_cancel_reason', 'Segment', 'Label']]

#Filter on the respective order_status
df_cobra = df_filter[df_filter['new_cancel_reason'] == 'COBRA']
df_ocara = df_filter[df_filter['new_cancel_reason'] == 'OCARA']
df_cobrm = df_filter[df_filter['new_cancel_reason'] == 'COBRM']
df_order_created = df_filter[df_filter['event_type'] == 'order_requested']
df_accepted = df_filter[df_filter['event_type'] == 'accepted']

#Sort the dfs by timestamp
df_cobra = df_cobra.sort_values(by = 'timestamp', ascending=True)
df_cobra = df_cobra.drop_duplicates(subset='order_id')

df_ocara = df_ocara.sort_values(by = 'timestamp', ascending=True)
df_ocara = df_ocara.drop_duplicates(subset='order_id')

df_cobrm = df_cobrm.sort_values(by = 'timestamp', ascending=True)
df_cobrm = df_cobrm.drop_duplicates(subset='order_id')

df_accepted = df_accepted.sort_values(by = 'timestamp', ascending=True)
df_accepted = df_accepted.drop_duplicates(subset='order_id')

#Change the column names
df_cobra = df_cobra[['order_id', 'new_cancel_reason', 'timestamp']]
df_cobra.columns = ['order_id', 'COBRA', 'timestamp_cobra']

df_cobrm = df_cobrm[['order_id', 'new_cancel_reason', 'timestamp']]
df_cobrm.columns = ['order_id', 'COBRM', 'timestamp_cobrm']

df_ocara = df_ocara[['order_id', 'new_cancel_reason', 'timestamp']]
df_ocara.columns = ['order_id', 'OCARA', 'timestamp_ocara']

df_accepted = df_accepted[['order_id', 'event_type', 'timestamp']]
df_accepted.columns = ['order_id', 'Accepted', 'timestamp_accepted']

#Merge the dfs together
dfs_to_merge = [df_order_created, df_cobra, df_cobrm, df_ocara, df_accepted]
df_final = reduce(lambda  left, right: pd.merge(left, right,on=['order_id'], how='left'), dfs_to_merge).fillna('NA')
df_final.head()

,order_id,pickup_location_hex_8,hour,yyyymmdd,timestamp,event_type,new_cancel_reason,Segment,Label,COBRA,timestamp_cobra,COBRM,timestamp_cobrm,OCARA,timestamp_ocara,Accepted,timestamp_accepted
0,644122381c6d14472f498524,88618925d3fffff,17,20230420,2023-04-20 17:00:00.725,order_requested,no_cancellation,out-high,Zone,NA,NA,NA,NA,NA,NA,NA,NA
1,6441223959513b224af3f391,8860145b49fffff,17,20230420,2023-04-20 17:00:01.911,order_requested,no_cancellation,out-high,Holdout,NA,NA,NA,NA,NA,NA,accepted,2023-04-20 17:00:19.384000
2,6441223a3382915e31cf0edd,88618925d9fffff,17,20230420,2023-04-20 17:00:02.293,order_requested,no_cancellation,balanced-high,Holdout,NA,NA,NA,NA,OCARA,2023-04-20 17:02:27.711000,accepted,2023-04-20 17:00:15.085000
3,6441223cf4689d1c09459f0e,886189219bfffff,17,20230420,2023-04-20 17:00:04.802,order_requested,no_cancellation,out-high,Holdout,NA,NA,NA,NA,NA,NA,NA,NA
4,6441223e1c6d14472f49852f,88618924e5fffff,17,20230420,2023-04-20 17:00:06.165,order_requested,no_cancellation,balanced-high,Zone,NA,NA,NA,NA,NA,NA,accepted,2023-04-20 17:00:12.150000


In [20]:
#Filter on the respective order_status
df_cobra = df_final[df_final['COBRA'] == 'COBRA']
df_ocara = df_final[df_final['OCARA'] == 'OCARA']
df_cobrm = df_final[df_final['COBRM'] == 'COBRM']
df_accepted = df_final[df_final['Accepted'] == 'accepted']

#Calculate TTC for cobrm, ocara, and cobra and tta
df_cobra['ttc_cobra'] = pd.to_datetime(df_cobra['timestamp_cobra']) - pd.to_datetime(df_cobra['timestamp'])
df_cobra['ttc_cobra'] = df_cobra['ttc_cobra'].apply(lambda x: x.total_seconds())

df_cobrm['ttc_cobrm'] = pd.to_datetime(df_cobrm['timestamp_cobrm']) - pd.to_datetime(df_cobrm['timestamp'])
df_cobrm['ttc_cobrm'] = df_cobrm['ttc_cobrm'].apply(lambda x: x.total_seconds())

df_ocara['ttc_ocara'] = pd.to_datetime(df_ocara['timestamp_ocara']) - pd.to_datetime(df_ocara['timestamp'])
df_ocara['ttc_ocara'] = df_ocara['ttc_ocara'].apply(lambda x: x.total_seconds())

df_accepted['tta'] = pd.to_datetime(df_accepted['timestamp_accepted']) - pd.to_datetime(df_accepted['timestamp'])
df_accepted['tta'] = df_accepted['tta'].apply(lambda x: x.total_seconds())

#Filter only on the new columns made
df_cobra = df_cobra[['order_id', 'ttc_cobra']]
df_cobrm = df_cobrm[['order_id', 'ttc_cobrm']]
df_ocara = df_ocara[['order_id', 'ttc_ocara']]
df_accepted = df_accepted[['order_id', 'tta']]

#Merge the dfs together
dfs_to_merge = [df_final, df_cobra, df_cobrm, df_ocara, df_accepted]
df_final = reduce(lambda  left, right: pd.merge(left, right,on=['order_id'], how='left'), dfs_to_merge)
df_final.head()

,order_id,pickup_location_hex_8,hour,yyyymmdd,timestamp,event_type,new_cancel_reason,Segment,Label,COBRA,timestamp_cobra,COBRM,timestamp_cobrm,OCARA,timestamp_ocara,Accepted,timestamp_accepted,ttc_cobra,ttc_cobrm,ttc_ocara,tta
0,644122381c6d14472f498524,88618925d3fffff,17,20230420,2023-04-20 17:00:00.725,order_requested,no_cancellation,out-high,Zone,NA,NA,NA,NA,NA,NA,NA,NA,NaN,NaN,NaN,NaN
1,6441223959513b224af3f391,8860145b49fffff,17,20230420,2023-04-20 17:00:01.911,order_requested,no_cancellation,out-high,Holdout,NA,NA,NA,NA,NA,NA,accepted,2023-04-20 17:00:19.384000,NaN,NaN,NaN,17.473
2,6441223a3382915e31cf0edd,88618925d9fffff,17,20230420,2023-04-20 17:00:02.293,order_requested,no_cancellation,balanced-high,Holdout,NA,NA,NA,NA,OCARA,2023-04-20 17:02:27.711000,accepted,2023-04-20 17:00:15.085000,NaN,NaN,145.418,12.792
3,6441223cf4689d1c09459f0e,886189219bfffff,17,20230420,2023-04-20 17:00:04.802,order_requested,no_cancellation,out-high,Holdout,NA,NA,NA,NA,NA,NA,NA,NA,NaN,NaN,NaN,NaN
4,6441223e1c6d14472f49852f,88618924e5fffff,17,20230420,2023-04-20 17:00:06.165,order_requested,no_cancellation,balanced-high,Zone,NA,NA,NA,NA,NA,NA,accepted,2023-04-20 17:00:12.150000,NaN,NaN,NaN,5.985


In [21]:
#Create a date column
df_final['date'] = df_final['yyyymmdd'].astype(str).apply(lambda x: x[6:])
df_final['count'] = 1

#Get average ttc, tta
df_cobra_grouped = df_final[df_final['COBRA'] == 'COBRA'].groupby(['yyyymmdd', 'hour', 'Segment', 'Label'])['ttc_cobra'].median().reset_index()
df_ocara_grouped = df_final[df_final['OCARA'] == 'OCARA'].groupby(['yyyymmdd', 'hour', 'Segment', 'Label'])['ttc_ocara'].median().reset_index()
df_cobrm_grouped = df_final[df_final['COBRM'] == 'COBRM'].groupby(['yyyymmdd', 'hour', 'Segment', 'Label'])['ttc_cobrm'].median().reset_index()
df_accepted_grouped = df_final[df_final['Accepted'] == 'accepted'].groupby(['yyyymmdd', 'hour', 'Segment', 'Label'])['tta'].median().reset_index()

#Merge the dfs together
dfs_to_merge = [df_cobra_grouped, df_cobrm_grouped, df_ocara_grouped, df_accepted_grouped]
df_metrics_final = reduce(lambda  left, right: pd.merge(left, right,on=['yyyymmdd', 'hour', 'Segment', 'Label'], how='left'), dfs_to_merge).fillna('NA')
df_metrics_final.head()

,yyyymmdd,hour,Segment,Label,ttc_cobra,ttc_cobrm,ttc_ocara,tta
0,20230420,17,balanced-high,Holdout,85.917,NA,252.8895,12.2890
1,20230420,17,balanced-high,Zone,98.849,NA,213.4755,7.6840
2,20230420,17,balanced-low,Holdout,41.271,NA,267.631,12.9895
3,20230420,17,in-high,Holdout,90.962,29.114,306.376,9.2860
4,20230420,17,in-high,Zone,102.484,69.23,185.6225,10.4425


In [22]:
#Merge ttc with metrics_df
metrics_df = metrics_df.merge(df_metrics_final, on = ['yyyymmdd', 'hour', 'Segment', 'Label'], how = 'left')
metrics_df.head()

,yyyymmdd,hour,Segment,Label,OCARA%,COBRA%,COBRM%,G2N,expiry_mapped%,Stockout%,total,FM,eta,number_of_captains,ttc_cobra,ttc_cobrm,ttc_ocara,tta
0,20230420,17,balanced-high,Holdout,0.0778,0.3346,0.0000,0.4747,0.1128,0.0000,257,24.310947,3.070039,46.5,85.917,NA,252.8895,12.2890
1,20230420,17,balanced-high,Zone,0.1168,0.1825,0.0000,0.6095,0.0876,0.0000,274,27.224927,2.722628,72.0,98.849,NA,213.4755,7.6840
2,20230420,17,balanced-low,Holdout,0.1373,0.2745,0.0000,0.2549,0.3137,0.0000,51,1.125160,2.823529,13.0,41.271,NA,267.631,12.9895
3,20230420,17,in-high,Holdout,0.0954,0.2977,0.0016,0.3865,0.2138,0.0016,608,0.856027,2.861842,72.0,90.962,29.114,306.376,9.2860
4,20230420,17,in-high,Zone,0.1222,0.2379,0.0011,0.5080,0.1275,0.0021,933,9.423077,2.780279,139.0,102.484,69.23,185.6225,10.4425


In [23]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, captain_location_hex_8, captain_location_latitude,
captain_location_longitude, city_name, customer_location_hex_8, drop_location_hex_8, drop_location_latitude, drop_location_longitude, customer_location_latitude, 
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason, updated_epoch, event_type, captain_id, spd_fraud_flag, pickup_location_hex_8
from orders.order_logs_immutable
where yyyymmdd >= '20230420'
and yyyymmdd <= '20230501'
and service_obj_service_name = 'Auto'
and event_type in ('accepted', 'rider_busy', 'rider_reject', 'dropped')
and pickup_location_hex_8 in ('88618924bbfffff', '886189260bfffff', '8860145913fffff', '88618924b3fffff', '8860145823fffff', '8860169659fffff', '886014595dfffff',
 '8860145a07fffff', '8860145847fffff', '8860169607fffff', '8861892c81fffff', '88601451e1fffff', '8861892687fffff', '88618921e3fffff', '8861892187fffff',
   '88601451e5fffff', '8860145acdfffff', '88618920ebfffff', '886189203dfffff', '8860145a83fffff', '8860145a87fffff', '886189208dfffff', '8860169645fffff',
     '88601696dbfffff', '8861892debfffff', '8861892495fffff', '8860145125fffff', '8861892c3bfffff', '88618921c5fffff', '8860169621fffff', '8860169619fffff',
       '8860145aadfffff', '8860145921fffff', '88618934e3fffff', '886014c905fffff', '8860145907fffff', '88618921cdfffff', '8861892281fffff', '8860145101fffff',
         '8860145ab5fffff', '8861892117fffff', '8861893495fffff', '88601690c7fffff', '886016968dfffff', '8860145995fffff', '886189228dfffff', '886016944bfffff',
           '88618934b1fffff', '8861892f65fffff', '8860169687fffff', '8860169743fffff', '8860145115fffff', '8861892691fffff', '8860169655fffff', '8860145359fffff',
             '8861892461fffff', '8860145b3bfffff', '88618925ddfffff', '8861892427fffff', '8861892c4bfffff', '88618925e9fffff', '8861892433fffff', '8861892401fffff',
               '886189242bfffff', '8861892423fffff', '88618925c9fffff', '88618924e7fffff', '8861892437fffff', '8861892439fffff', '8861892447fffff',
                 '8861892415fffff', '8861892c53fffff', '8861892469fffff', '886189243bfffff', '886189246bfffff', '886189243dfffff', '8861892e03fffff',
                   '886189246dfffff', '88618925cbfffff', '8861892e33fffff', '886189240bfffff', '8861892403fffff', '8860145b43fffff', '8861892e5dfffff',
                     '8861892463fffff', '88618920adfffff', '88618924adfffff', '8861892563fffff', '8861892cd3fffff', '8861892721fffff', '8860145a59fffff',
                       '8861892429fffff', '8861892eb3fffff', '8861892e17fffff', '8861892e61fffff', '886189252bfffff', '8861892445fffff', '8861892431fffff',
                         '8861892e67fffff', '8860145b19fffff', '8860145a55fffff', '8861892603fffff', '886189240dfffff', '8861892519fffff', '8861892405fffff',
                           '8861892407fffff', '8861892451fffff', '8861892e05fffff', '8861892717fffff', '8860145945fffff', '8861892ee7fffff', '8861892e45fffff',
                             '8861892c09fffff', '8861892465fffff', '88618926c1fffff', '8861892085fffff', '8861892625fffff', '88618920d3fffff', '886016975bfffff',
                               '8861892c69fffff', '886014534dfffff', '8860144b43fffff', '8860145acbfffff', '886014592bfffff', '8861892cb5fffff', '8861892ca9fffff',
                                 '8861892da3fffff', '8860145133fffff', '8861892db9fffff', '88618926e1fffff', '8860145951fffff', '88601458c5fffff', '8860145831fffff',
                                   '886014c925fffff', '8861892dc5fffff', '8861892dc7fffff', '88618926d1fffff', '88618935b7fffff', '8861892467fffff',
                                     '8861892591fffff', '88618925d9fffff', '88618925dbfffff', '8861892425fffff', '88618924e5fffff', '88618924edfffff',
                                       '8861892e1bfffff', '8861892435fffff', '8860145b1dfffff', '88601451adfffff', '8860145365fffff', '886014c9adfffff',
                                         '88618922b3fffff', '8860145ac9fffff', '8861892ca5fffff', '886014536bfffff', '88618935b1fffff', '8860145b37fffff',
                                           '8860145919fffff', '886014c9a9fffff', '886189269dfffff', '8860145a15fffff', '88618934b3fffff', '8860144b4dfffff',
                                             '88601451e7fffff', '8861892697fffff', '886014c923fffff', '8860145aedfffff', '886014c915fffff', '8860145341fffff',
                                               '8860145117fffff', '8861892293fffff', '8860169719fffff', '8860145363fffff', '88601459b5fffff', '8860145ae1fffff',
                                                 '886014c92dfffff', '88601696e3fffff', '8861892c63fffff', '8860145113fffff', '8860144a6dfffff', '8860145315fffff',
                                                   '8860144b55fffff', '886014c937fffff', '8860145abbfffff', '8860145b4bfffff', '8861892e91fffff', '88618925d3fffff',
                                                     '8861892621fffff', '8860145b49fffff', '88618925d1fffff', '886189255bfffff', '8861892545fffff', '88618925d7fffff',
                                                       '8861892e0dfffff', '8861892421fffff', '886189258dfffff', '88618925b5fffff', '8861892553fffff',
                                                         '886189242dfffff', '88618925b7fffff', '8860145949fffff', '8861892e41fffff', '886189219bfffff',
                                                           '886016965dfffff', '8861892539fffff', '886189265bfffff', '8861892505fffff', '88618934a3fffff',
                                                             '8861892501fffff', '8860169681fffff')
and hour in ('17', '18', '19', '20', '21')
"""

#Load data into pandas table
df_apr_copy = pd.read_sql(q, connection)

In [24]:
#Load and view dataset
df_apr = df_apr_copy.copy()
df_apr = df_apr[df_apr['pickup_location_hex_8'].isin(hyderabad_new_zone)]
df_apr = df_apr.merge(hexes, on = 'pickup_location_hex_8', how = 'left')
df_apr.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,updated_epoch,event_type,captain_id,spd_fraud_flag,pickup_location_hex_8,Segment,Label
0,64416367d1e1ab77e4e6d38c,NaN,8861892599fffff,12.928534,77.586217,Bangalore,88618925d7fffff,88618925c9fffff,12.920247,77.620543,12.927902,77.595604,213807,21,2130,Auto,21:59,4,20230420,new,None,1682006970517,rider_reject,60f00a148f347964bf172909,None,88618925d7fffff,out-high,Holdout
1,6441638d246f7347ed4f5d74,NaN,8861892421fffff,12.911813,77.638123,Bangalore,8861892421fffff,88618925c7fffff,12.933806,77.615336,12.912021,77.636292,213845,21,2130,Auto,21:59,4,20230420,new,None,1682006955059,rider_reject,6239c6e2c9fed0772e5e2e98,None,8861892421fffff,out-high,Zone
2,6441639a2c59400224161ad2,NaN,8861892467fffff,12.900445,77.650353,Bangalore,886189255bfffff,8861892425fffff,12.915927,77.642888,12.909071,77.650612,213858,21,2130,Auto,21:59,4,20230420,new,None,1682006979745,rider_reject,61e11e592dcf8a094697461f,None,886189255bfffff,out-high,Holdout
3,644164427d231f22a88f0322,NaN,886189255bfffff,12.912291,77.651782,Bangalore,8861892553fffff,886189242bfffff,12.902437,77.633062,12.921543,77.649582,214146,21,2130,Auto,21:59,4,20230420,new,None,1682007157581,rider_reject,6198e7be3f25c32324e280ae,None,8861892553fffff,out-high,Zone
4,6441644d2c59400224161be2,NaN,8861892595fffff,12.941807,77.587048,Bangalore,8861892591fffff,8861892403fffff,12.888528,77.620885,12.935178,77.583992,214157,21,2130,Auto,21:59,4,20230420,new,None,1682007165546,rider_reject,6238865469d08767567f0f84,None,8861892591fffff,balanced-high,Holdout


In [25]:
#Create a count column
df_apr['count'] = 1

#Create a column to identify total pings and accepted ping
df_apr['total_pings'] = np.where(df_apr['event_type'].isin(['accepted', 'rider_busy', 'rider_reject']), 1, 0)
df_apr['accepted_pings'] = np.where(df_apr['event_type'].isin(['accepted']), 1, 0)
df_me = df_apr[(df_apr['order_status'] == 'dropped') & (df_apr['spd_fraud_flag'] != True)]

#Groupby on date and hour to get the APR and ME on an hourly basis
apr_hyderabad = df_apr.groupby(['yyyymmdd', 'hour', 'Segment', 'Label'])[['total_pings', 'accepted_pings']].sum().reset_index()
apr_hyderabad['APR'] = round(apr_hyderabad['accepted_pings']/apr_hyderabad['total_pings'], 4)
me_hyderabad = df_me.groupby(['yyyymmdd', 'hour', 'Segment', 'Label'])['order_id'].nunique().reset_index()
me_hyderabad = me_hyderabad.rename(columns={'order_id':'net_orders'})
apr_hyderabad = apr_hyderabad.merge(me_hyderabad, on = ['yyyymmdd', 'hour', 'Segment', 'Label'], how = 'left')
apr_hyderabad['ME'] = round(apr_hyderabad['total_pings']/apr_hyderabad['net_orders'], 2)

#Merge APR and ME with metrics_df
metrics_df = metrics_df.merge(apr_hyderabad, on = ['yyyymmdd', 'hour', 'Segment', 'Label'], how = 'left')
metrics_df.head()

,yyyymmdd,hour,Segment,Label,OCARA%,COBRA%,COBRM%,G2N,expiry_mapped%,Stockout%,total,FM,eta,number_of_captains,ttc_cobra,ttc_cobrm,ttc_ocara,tta,total_pings,accepted_pings,APR,net_orders,ME
0,20230420,17,balanced-high,Holdout,0.0778,0.3346,0.0000,0.4747,0.1128,0.0000,257,24.310947,3.070039,46.5,85.917,NA,252.8895,12.2890,3276,179,0.0546,122,26.85
1,20230420,17,balanced-high,Zone,0.1168,0.1825,0.0000,0.6095,0.0876,0.0000,274,27.224927,2.722628,72.0,98.849,NA,213.4755,7.6840,3329,223,0.0670,167,19.93
2,20230420,17,balanced-low,Holdout,0.1373,0.2745,0.0000,0.2549,0.3137,0.0000,51,1.125160,2.823529,13.0,41.271,NA,267.631,12.9895,748,29,0.0388,13,57.54
3,20230420,17,in-high,Holdout,0.0954,0.2977,0.0016,0.3865,0.2138,0.0016,608,0.856027,2.861842,72.0,90.962,29.114,306.376,9.2860,8077,372,0.0461,233,34.67
4,20230420,17,in-high,Zone,0.1222,0.2379,0.0011,0.5080,0.1275,0.0021,933,9.423077,2.780279,139.0,102.484,69.23,185.6225,10.4425,13321,727,0.0546,471,28.28


In [26]:
#Query data to calculate gsr
q = """
select 
        pickup_location_hex_8, yyyymmdd, hour,
        count(distinct order_id) as Gross_Orders,
        count(case when (time_diff >=20 or time_diff is NULL) then customer_id end) as GSR,
        count(distinct case when event_type in ('dropped') and spd_fraud_flag != True then order_id end) as net_orders,
        (case when count(distinct case when event_type in ('dropped') and spd_fraud_flag != True then order_id end) = 0 then 0 else
        sum(case when event_type in ('accepted', 'rider_busy', 'rider_reject') then 1 else 0 end)
        /cast(count(distinct case when event_type in ('dropped') and spd_fraud_flag != True then order_id end) as real) end) as ME
from
(
select
        *, 
        date_diff('minute',  LAG(FROM_UNIXTIME(CAST(epoch AS double) /1000)) OVER(PARTITION BY yyyymmdd, hour, city_name, customer_id, order_type, customer_location_hex_8
        ORDER BY yyyymmdd, customer_id, city_name, customer_location_hex_8, order_type, epoch), FROM_UNIXTIME(CAST(epoch AS double) /1000)) as time_diff,
        date_diff('minute',  LAG(FROM_UNIXTIME(CAST(epoch AS double) /1000)) OVER(PARTITION BY yyyymmdd, hour, city_name, customer_id, order_type, order_status, customer_location_hex_8, spd_fraud_flag 
        ORDER BY yyyymmdd, customer_id, city_name, customer_location_hex_8, order_type, epoch) , FROM_UNIXTIME(CAST(epoch AS double) /1000)) as time_diff_drop
from
(
select 
    order_id, city_name, order_type, cancel_reason, yyyymmdd, customer_id, epoch, substr(hhmmss,1,2) as hour,
    row_number() over (partition by order_id order by updated_epoch desc) as Rank_1,
    
    event_type, order_status, captain_id, pickup_cluster, customer_location_hex_8, spd_fraud_flag, pickup_location_hex_8

from
    orders.order_logs_immutable 
where
    yyyymmdd >= '20230420'
    and yyyymmdd <= '20230501'
    and service_obj_service_name IN ('Auto')
    and pickup_location_hex_8 in ('88618924bbfffff', '886189260bfffff', '8860145913fffff', '88618924b3fffff', '8860145823fffff', '8860169659fffff', '886014595dfffff',
 '8860145a07fffff', '8860145847fffff', '8860169607fffff', '8861892c81fffff', '88601451e1fffff', '8861892687fffff', '88618921e3fffff', '8861892187fffff',
   '88601451e5fffff', '8860145acdfffff', '88618920ebfffff', '886189203dfffff', '8860145a83fffff', '8860145a87fffff', '886189208dfffff', '8860169645fffff',
     '88601696dbfffff', '8861892debfffff', '8861892495fffff', '8860145125fffff', '8861892c3bfffff', '88618921c5fffff', '8860169621fffff', '8860169619fffff',
       '8860145aadfffff', '8860145921fffff', '88618934e3fffff', '886014c905fffff', '8860145907fffff', '88618921cdfffff', '8861892281fffff', '8860145101fffff',
         '8860145ab5fffff', '8861892117fffff', '8861893495fffff', '88601690c7fffff', '886016968dfffff', '8860145995fffff', '886189228dfffff', '886016944bfffff',
           '88618934b1fffff', '8861892f65fffff', '8860169687fffff', '8860169743fffff', '8860145115fffff', '8861892691fffff', '8860169655fffff', '8860145359fffff',
             '8861892461fffff', '8860145b3bfffff', '88618925ddfffff', '8861892427fffff', '8861892c4bfffff', '88618925e9fffff', '8861892433fffff', '8861892401fffff',
               '886189242bfffff', '8861892423fffff', '88618925c9fffff', '88618924e7fffff', '8861892437fffff', '8861892439fffff', '8861892447fffff',
                 '8861892415fffff', '8861892c53fffff', '8861892469fffff', '886189243bfffff', '886189246bfffff', '886189243dfffff', '8861892e03fffff',
                   '886189246dfffff', '88618925cbfffff', '8861892e33fffff', '886189240bfffff', '8861892403fffff', '8860145b43fffff', '8861892e5dfffff',
                     '8861892463fffff', '88618920adfffff', '88618924adfffff', '8861892563fffff', '8861892cd3fffff', '8861892721fffff', '8860145a59fffff',
                       '8861892429fffff', '8861892eb3fffff', '8861892e17fffff', '8861892e61fffff', '886189252bfffff', '8861892445fffff', '8861892431fffff',
                         '8861892e67fffff', '8860145b19fffff', '8860145a55fffff', '8861892603fffff', '886189240dfffff', '8861892519fffff', '8861892405fffff',
                           '8861892407fffff', '8861892451fffff', '8861892e05fffff', '8861892717fffff', '8860145945fffff', '8861892ee7fffff', '8861892e45fffff',
                             '8861892c09fffff', '8861892465fffff', '88618926c1fffff', '8861892085fffff', '8861892625fffff', '88618920d3fffff', '886016975bfffff',
                               '8861892c69fffff', '886014534dfffff', '8860144b43fffff', '8860145acbfffff', '886014592bfffff', '8861892cb5fffff', '8861892ca9fffff',
                                 '8861892da3fffff', '8860145133fffff', '8861892db9fffff', '88618926e1fffff', '8860145951fffff', '88601458c5fffff', '8860145831fffff',
                                   '886014c925fffff', '8861892dc5fffff', '8861892dc7fffff', '88618926d1fffff', '88618935b7fffff', '8861892467fffff',
                                     '8861892591fffff', '88618925d9fffff', '88618925dbfffff', '8861892425fffff', '88618924e5fffff', '88618924edfffff',
                                       '8861892e1bfffff', '8861892435fffff', '8860145b1dfffff', '88601451adfffff', '8860145365fffff', '886014c9adfffff',
                                         '88618922b3fffff', '8860145ac9fffff', '8861892ca5fffff', '886014536bfffff', '88618935b1fffff', '8860145b37fffff',
                                           '8860145919fffff', '886014c9a9fffff', '886189269dfffff', '8860145a15fffff', '88618934b3fffff', '8860144b4dfffff',
                                             '88601451e7fffff', '8861892697fffff', '886014c923fffff', '8860145aedfffff', '886014c915fffff', '8860145341fffff',
                                               '8860145117fffff', '8861892293fffff', '8860169719fffff', '8860145363fffff', '88601459b5fffff', '8860145ae1fffff',
                                                 '886014c92dfffff', '88601696e3fffff', '8861892c63fffff', '8860145113fffff', '8860144a6dfffff', '8860145315fffff',
                                                   '8860144b55fffff', '886014c937fffff', '8860145abbfffff', '8860145b4bfffff', '8861892e91fffff', '88618925d3fffff',
                                                     '8861892621fffff', '8860145b49fffff', '88618925d1fffff', '886189255bfffff', '8861892545fffff', '88618925d7fffff',
                                                       '8861892e0dfffff', '8861892421fffff', '886189258dfffff', '88618925b5fffff', '8861892553fffff',
                                                         '886189242dfffff', '88618925b7fffff', '8860145949fffff', '8861892e41fffff', '886189219bfffff',
                                                           '886016965dfffff', '8861892539fffff', '886189265bfffff', '8861892505fffff', '88618934a3fffff',
                                                             '8861892501fffff', '8860169681fffff')
and hour in ('17', '18', '19', '20', '21')
))
group by 1,2,3
"""

df_gsr_copy = pd.read_sql(q, connection)

In [27]:
#Create a copy of the dataset and view it
df_gsr = df_gsr_copy.copy()
df_gsr = df_gsr[df_gsr['pickup_location_hex_8'].isin(hyderabad_new_zone)]
df_gsr = df_gsr.merge(hexes, on = 'pickup_location_hex_8', how = 'left')
df_gsr.head()

,pickup_location_hex_8,yyyymmdd,hour,Gross_Orders,GSR,net_orders,ME,Segment,Label
0,88618925ddfffff,20230422,21,166,90,13,75.538460,in-high,Zone
1,88618924e5fffff,20230423,19,202,103,17,50.294117,balanced-high,Zone
2,8861892405fffff,20230427,19,55,37,13,62.384617,in-high,Zone
3,8860145b3bfffff,20230426,19,46,35,14,60.214287,in-high,Holdout
4,8861892553fffff,20230501,17,242,121,51,35.764706,out-high,Zone


In [28]:
#Calculate GSR2Net
df_gsr_grouped = df_gsr.groupby(['yyyymmdd', 'hour', 'Segment', 'Label'])[['GSR', 'net_orders']].sum().reset_index()
df_gsr_grouped['GSR2Net'] = round(df_gsr_grouped['net_orders']/df_gsr_grouped['GSR'], 4)

#Merge GSR with metrics_df
metrics_df = metrics_df.merge(df_gsr_grouped, on = ['yyyymmdd', 'hour', 'Segment', 'Label'], how = 'left')
metrics_df.head()

,yyyymmdd,hour,Segment,Label,OCARA%,COBRA%,COBRM%,G2N,expiry_mapped%,Stockout%,total,FM,eta,number_of_captains,ttc_cobra,ttc_cobrm,ttc_ocara,tta,total_pings,accepted_pings,APR,net_orders_x,ME,GSR,net_orders_y,GSR2Net
0,20230420,17,balanced-high,Holdout,0.0778,0.3346,0.0000,0.4747,0.1128,0.0000,257,24.310947,3.070039,46.5,85.917,NA,252.8895,12.2890,3276,179,0.0546,122,26.85,211,122,0.5782
1,20230420,17,balanced-high,Zone,0.1168,0.1825,0.0000,0.6095,0.0876,0.0000,274,27.224927,2.722628,72.0,98.849,NA,213.4755,7.6840,3329,223,0.0670,167,19.93,244,167,0.6844
2,20230420,17,balanced-low,Holdout,0.1373,0.2745,0.0000,0.2549,0.3137,0.0000,51,1.125160,2.823529,13.0,41.271,NA,267.631,12.9895,748,29,0.0388,13,57.54,33,13,0.3939
3,20230420,17,in-high,Holdout,0.0954,0.2977,0.0016,0.3865,0.2138,0.0016,608,0.856027,2.861842,72.0,90.962,29.114,306.376,9.2860,8077,372,0.0461,233,34.67,460,233,0.5065
4,20230420,17,in-high,Zone,0.1222,0.2379,0.0011,0.5080,0.1275,0.0021,933,9.423077,2.780279,139.0,102.484,69.23,185.6225,10.4425,13321,727,0.0546,471,28.28,768,471,0.6133


In [29]:
#Query data from supply_login_servicelevel_kpi
q = """
select
        location as pickup_location_hex_8, yyyymmdd, substr(hhmm, 1, 2) as hour,
        sum(case when cast(status AS varchar) IN ('2','3','6','7','8','10') then duration end) as login_time,
        sum(case when cast(status AS varchar) IN ('2') then duration end) as idle_time
from 
(select 
        a.*, city, cluster
        
from 
        hive.datasets.supplycursory_history a 
        left join datasets.city_cluster_hex b 
        on a.location = b.hex_id and a.resolution = cast(b.resolution as varchar)
where 
        yyyymmdd >= '20230420'
        and yyyymmdd <= '20230501'
        and city in ('Bangalore')
        and a.resolution = '8' 
        and service in ('auto')
        and location in ('88618924bbfffff', '886189260bfffff', '8860145913fffff', '88618924b3fffff', '8860145823fffff', '8860169659fffff', '886014595dfffff',
 '8860145a07fffff', '8860145847fffff', '8860169607fffff', '8861892c81fffff', '88601451e1fffff', '8861892687fffff', '88618921e3fffff', '8861892187fffff',
   '88601451e5fffff', '8860145acdfffff', '88618920ebfffff', '886189203dfffff', '8860145a83fffff', '8860145a87fffff', '886189208dfffff', '8860169645fffff',
     '88601696dbfffff', '8861892debfffff', '8861892495fffff', '8860145125fffff', '8861892c3bfffff', '88618921c5fffff', '8860169621fffff', '8860169619fffff',
       '8860145aadfffff', '8860145921fffff', '88618934e3fffff', '886014c905fffff', '8860145907fffff', '88618921cdfffff', '8861892281fffff', '8860145101fffff',
         '8860145ab5fffff', '8861892117fffff', '8861893495fffff', '88601690c7fffff', '886016968dfffff', '8860145995fffff', '886189228dfffff', '886016944bfffff',
           '88618934b1fffff', '8861892f65fffff', '8860169687fffff', '8860169743fffff', '8860145115fffff', '8861892691fffff', '8860169655fffff', '8860145359fffff',
             '8861892461fffff', '8860145b3bfffff', '88618925ddfffff', '8861892427fffff', '8861892c4bfffff', '88618925e9fffff', '8861892433fffff', '8861892401fffff',
               '886189242bfffff', '8861892423fffff', '88618925c9fffff', '88618924e7fffff', '8861892437fffff', '8861892439fffff', '8861892447fffff',
                 '8861892415fffff', '8861892c53fffff', '8861892469fffff', '886189243bfffff', '886189246bfffff', '886189243dfffff', '8861892e03fffff',
                   '886189246dfffff', '88618925cbfffff', '8861892e33fffff', '886189240bfffff', '8861892403fffff', '8860145b43fffff', '8861892e5dfffff',
                     '8861892463fffff', '88618920adfffff', '88618924adfffff', '8861892563fffff', '8861892cd3fffff', '8861892721fffff', '8860145a59fffff',
                       '8861892429fffff', '8861892eb3fffff', '8861892e17fffff', '8861892e61fffff', '886189252bfffff', '8861892445fffff', '8861892431fffff',
                         '8861892e67fffff', '8860145b19fffff', '8860145a55fffff', '8861892603fffff', '886189240dfffff', '8861892519fffff', '8861892405fffff',
                           '8861892407fffff', '8861892451fffff', '8861892e05fffff', '8861892717fffff', '8860145945fffff', '8861892ee7fffff', '8861892e45fffff',
                             '8861892c09fffff', '8861892465fffff', '88618926c1fffff', '8861892085fffff', '8861892625fffff', '88618920d3fffff', '886016975bfffff',
                               '8861892c69fffff', '886014534dfffff', '8860144b43fffff', '8860145acbfffff', '886014592bfffff', '8861892cb5fffff', '8861892ca9fffff',
                                 '8861892da3fffff', '8860145133fffff', '8861892db9fffff', '88618926e1fffff', '8860145951fffff', '88601458c5fffff', '8860145831fffff',
                                   '886014c925fffff', '8861892dc5fffff', '8861892dc7fffff', '88618926d1fffff', '88618935b7fffff', '8861892467fffff',
                                     '8861892591fffff', '88618925d9fffff', '88618925dbfffff', '8861892425fffff', '88618924e5fffff', '88618924edfffff',
                                       '8861892e1bfffff', '8861892435fffff', '8860145b1dfffff', '88601451adfffff', '8860145365fffff', '886014c9adfffff',
                                         '88618922b3fffff', '8860145ac9fffff', '8861892ca5fffff', '886014536bfffff', '88618935b1fffff', '8860145b37fffff',
                                           '8860145919fffff', '886014c9a9fffff', '886189269dfffff', '8860145a15fffff', '88618934b3fffff', '8860144b4dfffff',
                                             '88601451e7fffff', '8861892697fffff', '886014c923fffff', '8860145aedfffff', '886014c915fffff', '8860145341fffff',
                                               '8860145117fffff', '8861892293fffff', '8860169719fffff', '8860145363fffff', '88601459b5fffff', '8860145ae1fffff',
                                                 '886014c92dfffff', '88601696e3fffff', '8861892c63fffff', '8860145113fffff', '8860144a6dfffff', '8860145315fffff',
                                                   '8860144b55fffff', '886014c937fffff', '8860145abbfffff', '8860145b4bfffff', '8861892e91fffff', '88618925d3fffff',
                                                     '8861892621fffff', '8860145b49fffff', '88618925d1fffff', '886189255bfffff', '8861892545fffff', '88618925d7fffff',
                                                       '8861892e0dfffff', '8861892421fffff', '886189258dfffff', '88618925b5fffff', '8861892553fffff',
                                                         '886189242dfffff', '88618925b7fffff', '8860145949fffff', '8861892e41fffff', '886189219bfffff',
                                                           '886016965dfffff', '8861892539fffff', '886189265bfffff', '8861892505fffff', '88618934a3fffff',
                                                             '8861892501fffff', '8860169681fffff')
and substr(hhmm, 1, 2) in ('17', '18', '19', '20', '21')
)
group by 1,2,3
"""

#Load data into pandas table
df_tse_copy = pd.read_sql(q, connection)

In [30]:
#Create a copy of the df and view it
df_tse = df_tse_copy.copy()
df_tse = df_tse[df_tse['pickup_location_hex_8'].isin(hyderabad_new_zone)]
df_tse = df_tse.merge(hexes, on = 'pickup_location_hex_8', how = 'left')
df_tse.head()

,pickup_location_hex_8,yyyymmdd,hour,login_time,idle_time,Segment,Label
0,8860145823fffff,20230501,21,6797.0,3060.0,in-low,Holdout
1,8860169619fffff,20230423,21,20400.0,11506.0,in-low,Holdout
2,8860145a55fffff,20230423,17,29884.0,4993.0,in-high,Holdout
3,8860145a55fffff,20230501,19,14520.0,971.0,in-high,Holdout
4,8861892e33fffff,20230426,19,20315.0,2491.0,in-high,Holdout


In [31]:
#Calculate time_spent_earning
df_tse_grouped = df_tse.groupby(['yyyymmdd', 'hour', 'Segment', 'Label'])[['login_time', 'idle_time']].sum().reset_index()
df_tse_grouped['time_spent_earning'] = round((df_tse_grouped['login_time'] - df_tse_grouped['idle_time'])/df_tse_grouped['login_time'], 4)
df_tse_grouped.head()

,yyyymmdd,hour,Segment,Label,login_time,idle_time,time_spent_earning
0,20230420,17,balanced-high,Holdout,375497.0,96601.0,0.7427
1,20230420,17,balanced-high,Zone,520355.0,149685.0,0.7123
2,20230420,17,balanced-low,Holdout,188377.0,140732.0,0.2529
3,20230420,17,in-high,Holdout,1454735.0,625245.0,0.5702
4,20230420,17,in-high,Zone,2027193.0,704981.0,0.6522


In [32]:
#Merge GSR with metrics_df
metrics_df = metrics_df.merge(df_tse_grouped, on = ['yyyymmdd', 'hour', 'Segment', 'Label'], how = 'left')
metrics_df.head()

,yyyymmdd,hour,Segment,Label,OCARA%,COBRA%,COBRM%,G2N,expiry_mapped%,Stockout%,total,FM,eta,number_of_captains,ttc_cobra,ttc_cobrm,ttc_ocara,tta,total_pings,accepted_pings,APR,net_orders_x,ME,GSR,net_orders_y,GSR2Net,login_time,idle_time,time_spent_earning
0,20230420,17,balanced-high,Holdout,0.0778,0.3346,0.0000,0.4747,0.1128,0.0000,257,24.310947,3.070039,46.5,85.917,NA,252.8895,12.2890,3276,179,0.0546,122,26.85,211,122,0.5782,375497.0,96601.0,0.7427
1,20230420,17,balanced-high,Zone,0.1168,0.1825,0.0000,0.6095,0.0876,0.0000,274,27.224927,2.722628,72.0,98.849,NA,213.4755,7.6840,3329,223,0.0670,167,19.93,244,167,0.6844,520355.0,149685.0,0.7123
2,20230420,17,balanced-low,Holdout,0.1373,0.2745,0.0000,0.2549,0.3137,0.0000,51,1.125160,2.823529,13.0,41.271,NA,267.631,12.9895,748,29,0.0388,13,57.54,33,13,0.3939,188377.0,140732.0,0.2529
3,20230420,17,in-high,Holdout,0.0954,0.2977,0.0016,0.3865,0.2138,0.0016,608,0.856027,2.861842,72.0,90.962,29.114,306.376,9.2860,8077,372,0.0461,233,34.67,460,233,0.5065,1454735.0,625245.0,0.5702
4,20230420,17,in-high,Zone,0.1222,0.2379,0.0011,0.5080,0.1275,0.0021,933,9.423077,2.780279,139.0,102.484,69.23,185.6225,10.4425,13321,727,0.0546,471,28.28,768,471,0.6133,2027193.0,704981.0,0.6522


In [33]:
#Create a copy of the dataset and view it
df_captains_mapped = df_copy.copy()
df_captains_mapped = df_captains_mapped[df_captains_mapped['pickup_location_hex_8'].isin(hyderabad_new_zone)]
df_captains_mapped = df_captains_mapped.merge(hexes, on = 'pickup_location_hex_8', how = 'left')
df_captains_mapped.head()

,order_id,accept_to_pickup_distance,city_name,hhmmss,hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,map_riders,pickup_location_hex_8,Segment,Label
0,6443e20d306023341f53bb59,NaN,Bangalore,190301,19,Auto,19:00,6,20230422,expired,None,0.0,"[""5b55bb848b7b372a6fcf5832"",""60e808b7569b260e1...",8861892463fffff,in-high,Zone
1,6443e2175028ee299f9b6229,NaN,Bangalore,190311,19,Auto,19:00,6,20230422,customerCancelled,order cancelled before rider accepted,5.0,"[""61c2fb2017dd6a517a1b00c9"",""63da55eefbbdfd41f...",886189242bfffff,in-high,Zone
2,6443e2784ec9443f1ebb5fe3,NaN,Bangalore,190448,19,Auto,19:00,6,20230422,expired,None,7.0,"[""61c2fb2017dd6a517a1b00c9"",""638af2bea92ee651e...",8861892423fffff,in-high,Zone
3,6443e2840f2b53161ffe8f62,NaN,Bangalore,190500,19,Auto,19:00,6,20230422,expired,None,0.0,"[""641e8c52b0ac5a0e04598ddf"",""6334207c7248a1ab0...",8861892423fffff,in-high,Zone
4,6443e29e5028ee299f9b6366,NaN,Bangalore,190526,19,Auto,19:00,6,20230422,expired,None,0.0,"[""60e808b7569b260e1774330a"",""60ed4259ac6ff4537...",8861892463fffff,in-high,Zone


In [34]:
#Get the number of captains mapped
df_captains_mapped['map_riders'] = df_captains_mapped['map_riders'].apply(lambda x: json.loads(x))
df_captains_mapped_grouped = df_captains_mapped.groupby(['yyyymmdd', 'hour', 'Segment', 'Label'])['map_riders'].apply(list).reset_index()
df_captains_mapped_grouped['map_riders'] = df_captains_mapped_grouped['map_riders'].apply(lambda x: list(set([item for sublist in x for item in sublist])))
master_list = []
for x in df_captains_mapped_grouped['map_riders']:
    captains_list = []
    for captain in x:
        if captain in captains_list:
            continue
        else:
            captains_list.append(captain)
    master_list.append(captains_list)
df_captains_mapped_grouped['map_riders'] = master_list
df_captains_mapped_grouped['number_of_mapped_captains'] = df_captains_mapped_grouped['map_riders'].apply(lambda x: len(x))
metrics_df = metrics_df.merge(df_captains_mapped_grouped[['yyyymmdd', 'hour', 'Segment', 'Label', 'number_of_mapped_captains']], 
on = ['yyyymmdd', 'hour', 'Segment', 'Label'], how = 'left')
metrics_df.head()

,yyyymmdd,hour,Segment,Label,OCARA%,COBRA%,COBRM%,G2N,expiry_mapped%,Stockout%,total,FM,eta,number_of_captains,ttc_cobra,ttc_cobrm,ttc_ocara,tta,total_pings,accepted_pings,APR,net_orders_x,ME,GSR,net_orders_y,GSR2Net,login_time,idle_time,time_spent_earning,number_of_mapped_captains
0,20230420,17,balanced-high,Holdout,0.0778,0.3346,0.0000,0.4747,0.1128,0.0000,257,24.310947,3.070039,46.5,85.917,NA,252.8895,12.2890,3276,179,0.0546,122,26.85,211,122,0.5782,375497.0,96601.0,0.7427,1290
1,20230420,17,balanced-high,Zone,0.1168,0.1825,0.0000,0.6095,0.0876,0.0000,274,27.224927,2.722628,72.0,98.849,NA,213.4755,7.6840,3329,223,0.0670,167,19.93,244,167,0.6844,520355.0,149685.0,0.7123,1525
2,20230420,17,balanced-low,Holdout,0.1373,0.2745,0.0000,0.2549,0.3137,0.0000,51,1.125160,2.823529,13.0,41.271,NA,267.631,12.9895,748,29,0.0388,13,57.54,33,13,0.3939,188377.0,140732.0,0.2529,345
3,20230420,17,in-high,Holdout,0.0954,0.2977,0.0016,0.3865,0.2138,0.0016,608,0.856027,2.861842,72.0,90.962,29.114,306.376,9.2860,8077,372,0.0461,233,34.67,460,233,0.5065,1454735.0,625245.0,0.5702,3665
4,20230420,17,in-high,Zone,0.1222,0.2379,0.0011,0.5080,0.1275,0.0021,933,9.423077,2.780279,139.0,102.484,69.23,185.6225,10.4425,13321,727,0.0546,471,28.28,768,471,0.6133,2027193.0,704981.0,0.6522,2164


In [35]:
metrics_df.to_csv('metrics_holdout_hsr_lower.csv', index = False)